## Implicit feedbackを想定し、バイアスを考慮した推薦　　
### データが増えるとその分どのくらいバイアスに気を使わないといけないのか調査
人気順推薦から生まれるランキングバイアスパラメータを特定し、IPS推定量とNaive推定量を比較  
IPS推定量の分散が大きければ、ロバスト推定量を使用したいところ

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [101]:
import random

class train_test_split():
    
    def __init__(self) -> None:
        self.feature = []
        self.item_id = []
        self.populality = []

    def make_dataframe(self) -> pd.DataFrame:
        item_dict = {"item_id": self.item_id, "feature": self.feature, "populality": self.populality}
        df = pd.DataFrame(data=item_dict)
        
        return df
        
    def create_dataset(self) -> pd.DataFrame, pd.DataFrame:
    
        for i in range(100):
            self.feature.append(round(random.uniform(1,5),2))
            self.populality.append(round(random.uniform(1,5),2))
            self.item_id.append(str(i+1))
        
            if i == 69:
                df_train = self.make_dataframe()
                self.feature, self.item_id, self.populality = [], [], []
        
        df_test = self.make_dataframe()
        
        return df_train, df_test
                

# 特徴量の説明
feature:  ユーザーのクリックを決める特徴量(5に近いとクリックしやすい)  
populality:  一ヶ月ごとの人気度を表す特徴量(5に近いと推薦されやすい。人気順の推薦を想定)  
また、train, testは7:3

In [102]:
df, df_test = train_test_split().create_dataset()

In [104]:
df

,item_id,feature,populality
0,1,3.54,4.27
1,2,3.39,1.62
2,3,2.38,1.88
3,4,1.46,2.06
4,5,3.55,4.73
...,...,...,...
65,66,4.05,2.22
66,67,4.30,2.53
67,68,2.72,3.05
68,69,2.94,2.42


In [105]:
#人気順推薦でランキングしてみる（毎月のバッチ処理と仮定）
# 数字を高い順にすると、ログが同じになるので、少しランダムに
sorted_df = df.copy()
sorted_df = sorted_df[sorted_df["populality"] >= 3.8]
sorted_df.reset_index(drop=True, inplace=True)

In [106]:
# ポジションバイアスはE[O(u,i)]としpow_trueパラメータを0.5とする
# C(u,i,k) = O(u,i)*R(u,i)
# E[C(u,i,k)] = THETA(u,i)*r(u,i)

position_bias = lambda k: pow(0.9/k, 0.2) 

In [6]:
# r(u,i)をfeatureとし、0-1スケールに変換。featureはあくまで特徴量なので、
#誤クリックを加えることで過学習を防ぐ

In [107]:
def user_feedback() -> pd.DataFrame:
    position = list(range(1,8))
    item_dict = {
        "position": [], "item_id": [], 
        "clicked": [], "iter": []
    }
    for i in range(50):
        recommend_items = random.sample(list(sorted_df.index.values), k=7)
        df = sorted_df.loc[recommend_items,:]
        click_true = np.zeros(7);
        rel_max = 5
        rel = 0.1 + 0.9 * pow(2, df["feature"].values-1)/pow(2, rel_max-1)
    
        for k in range(len(click_true)):
            theta = position_bias(k+1)
            E_click = rel[k]*theta
            if (E_click >= 0.6):
                click_true[k] = 1
            else:
                # 誤クリック
                if (random.random() < 0.1):
                    click_true[k] = 1
        
        item_dict["position"].extend(position)
        item_dict["item_id"].extend(df["item_id"].values)
        item_dict["clicked"].extend(click_true)
        item_dict["iter"].extend(len(position)*[i+1])
    
    log_df = pd.DataFrame(data=item_dict)
    log_df = log_df.astype({"clicked": np.int64})
    
    count_dict = log_df[log_df["clicked"]==1].groupby("item_id").agg({"clicked": "count"})["clicked"].to_dict()
    history_dict = {"item_id": list(count_dict.keys()), "click count": list(count_dict.values())}
    user_df = pd.DataFrame(data=history_dict)
    
    return log_df, user_df

In [108]:
log_df, user_df = user_feedback()

In [111]:
log_df

,position,item_id,clicked,iter
0,1,50,0,1
1,2,5,0,1
2,3,41,0,1
3,4,53,1,1
4,5,27,0,1
...,...,...,...,...
345,3,23,1,50
346,4,21,0,50
347,5,50,0,50
348,6,42,0,50


In [110]:
df[df["item_id"].isin(user_df["item_id"].values)]

,item_id,feature,populality
0,1,3.54,4.27
4,5,3.55,4.73
12,13,4.38,3.82
20,21,1.18,4.60
21,22,4.41,4.45
22,23,4.97,4.05
24,25,4.51,4.04
26,27,1.23,4.38
27,28,3.88,4.03
34,35,4.63,4.85
